Imports

In [16]:
import re
from os import walk
import os
from urllib.request import urlopen
from lxml import html
from bs4 import BeautifulSoup
from urllib.request import urlopen
import urllib
from multiprocessing.pool import ThreadPool

Finding files

In [2]:
# Looking for the files in the /data folder
mypath = os.path.abspath(os.getcwd()) + "/data"
ext = "txt"
files = []
for (dirpath, dirnames, filenames) in walk(mypath):
    for file in filenames:
        if file.split(".")[1] == ext:  # Only files with the right extension
            files.append(file)
    break
if not files:  # if files is empty
    raise Exception("Warning there are no files, check the path!")
    quit()  # no need to proceed

s = "" if len(files) == 1 else "s"
print(f"Analyzing {len(files)} file{s}:")
[print("\t" + file) for file in files]

Analyzing 1 file:
	comp220.txt


[None]

Generating list of links from techleap

In [20]:
links = []

for file in files:
	file = open(f"Data/{file}", "r").read()
	for i in [m.start() for m in re.finditer('href', file)]:
		closingcommas = file[i : i+50].find(">")
		link = file[i+6 : i+closingcommas-1]
		links.append(link)
    file.close()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 9)

Generating list of companies links

In [14]:
complinks = []
nocomplinks = []
for link in links:
	url = "https://finder.techleap.nl" + link
	page = urlopen(url)
	soup = BeautifulSoup(page, 'html.parser')
	
	link = link.replace("_", "")
	save = True
	haslink = False
	for href in soup.find_all('a'):
		templink = href.get('href')
		if link.split("/")[2] in templink and "http" in templink:
			for domain in ["facebook", "google", "linkedin", "twitter", "apple"]:
				if domain in templink:
					save = False
			if save:
				print(link, "\t", templink)
				complinks.append(templink)
				haslink = True
	if not haslink:
		nocomplinks.append(link) 

/companies/mx3d 	 http://www.mx3d.com
/companies/getmibo 	 https://getmibo.com/
/companies/protifarm 	 https://protifarm.com/
/companies/shypple 	 http://www.shypple.com
/companies/swipeguide 	 http://www.swipeguide.com
/companies/inuka 	 http://inuka.io
/companies/24sessions 	 http://24sessions.com
/companies/lavatherapeutics 	 https://lavatherapeutics.com/
/companies/leydenlabs 	 http://www.leydenlabs.com
/companies/bringly 	 https://www.bringly.nl/
/companies/roadmap 	 http://www.getroadmap.com/
/companies/clairify 	 http://clairify.io
/companies/lightyear 	 https://www.lightyear.one/
/companies/fangage 	 http://www.fangage.com/
/companies/hyzonmotors 	 https://www.hyzonmotors.com/
/companies/sonicbee 	 https://www.sonicbee.nl/en/
/companies/founda 	 https://foundahealth.com/
/companies/postoplan 	 http://postoplan.app
/companies/veylinx 	 https://www.veylinx.com/
/companies/katanox 	 https://www.katanox.com
/companies/felyx 	 https://felyx.com/
/companies/hubx 	 https://hubx.capita

In [15]:
print(f"{len(links) - len(complinks)} didn't have a link out of {len(links)} companies")

41 didn't have a link out of 117 companies


Searching for emails

In [10]:
compmails = {}
for link in complinks:
    try:
        req = urllib.request.Request(link, headers={'User-Agent' : "Magic Browser"}) 
        con = urlopen(req)
    except Exception as e:
        print(f" * * * Error opening {link}")
        print(" * * * " + str(e))

    s = con.read().decode("utf-8")
    emails = re.findall(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,4}", s)
    newemails = list(set(emails))
    compmails[link] = []
    for mail in newemails:
        if mail.split(".")[-1] == link.split(".")[-1] and "%20" not in mail:
            compmails[link].append(mail)
            print(link, "\t", mail)

http://www.mx3d.com 	 info@mx3d.com
http://www.mx3d.com 	 press@mx3d.com
http://www.shypple.com 	 info@shypple.com
http://www.swipeguide.com 	 hello@swipeguide.com
http://clairify.io 	 felix@clairify.io
http://clairify.io 	 tibor@clairify.io
http://clairify.io 	 wim@clairify.io
http://clairify.io 	 info@clairify.io
https://www.katanox.com 	 hello@katanox.com
http://baqme.com 	 support@baqme.com
 * * * Error opening https://www.denoize.com
 * * * <urlopen error [SSL: TLSV1_ALERT_INTERNAL_ERROR] tlsv1 alert internal error (_ssl.c:1091)>
https://www.targedbiopharmaceuticals.com 	 info@targedbio.com
http://getrodeo.io 	 info@getrodeo.io
http://www.flego.nl 	 mathijs@flego.nl
http://www.flego.nl 	 info@flego.nl
https://enzyre.com 	 investors@enzyre.com
https://enzyre.com 	 info@enzyre.com
http://zielwear.com 	 email@address.com
http://zielwear.com 	 sayhi@zielwear.com
https://www.leadinfo.com 	 hello@leadinfo.com


In [11]:
for i in compmails:
    print(i)
    [print("\t" + x) for x in compmails[i]]

http://www.mx3d.com
	info@mx3d.com
	press@mx3d.com
http://www.shypple.com
	info@shypple.com
http://www.swipeguide.com
	hello@swipeguide.com
http://inuka.io
http://24sessions.com
https://www.bringly.nl/
http://www.getroadmap.com/
http://clairify.io
	felix@clairify.io
	tibor@clairify.io
	wim@clairify.io
	info@clairify.io
https://www.lightyear.one/
http://www.fangage.com/
https://www.sonicbee.nl/en/
https://foundahealth.com/
http://postoplan.app
https://www.veylinx.com/
https://www.katanox.com
	hello@katanox.com
https://felyx.com/
http://baqme.com
	support@baqme.com
http://tripactions.com
https://www.denoize.com
https://www.dockrmobility.nl/nl/
https://axoniq.io/
https://packaly.com/
http://www.omnigen.nl
https://www.confocal.nl/
https://www.cortexter.com
https://www.targedbiopharmaceuticals.com
	info@targedbio.com
https://satelligence.com/
http://getrodeo.io
	info@getrodeo.io
https://mriguidance.com/
https://viteezy.nl
https://weareeves.com/
https://syncvr.tech
https://www.medvice.io/
ht

In [17]:
for i in nocomplinks:
    print(i)

/companies/qlayersbv
/companies/kaminarimedicalbv
/companies/carttech
/companies/theselectionlabbv
/companies/invisiblefood
/companies/getstreamio
/companies/semioticlabs
/companies/spikemobility
/companies/ellipsisdrive
/companies/emagy
/companies/lipocoatbv
/companies/stilbv
/companies/orangegames
/companies/stack2
/companies/evbiotech
/companies/asthmawarebv
/companies/stepmobility
/companies/cooperapp
/companies/praxasensebv
/companies/ecgexcellence
/companies/trunkrsbv
/companies/amflow
/companies/tretbox
/companies/salviabioelectronics
/companies/neogenetherapeutics
/companies/semitechnologies
/companies/eindhovenmedicalrobotics
/companies/skyned
/companies/hyzonmotorseurope
/companies/skiniveholdingbv
/companies/slamorthobv
/companies/akthelhealthcare
/companies/vicotherapeutics
/companies/synthesisinstitute
/companies/inocellsbv
/companies/inicarehealthtechnology
/companies/asyliadiagnostics
/companies/hemoclearbv
/companies/unilife
/companies/actpluzbv
/companies/surgeon


In [ ]:
# Printing mails
for i in compmails():
	print(i)
	[print("\t" + x) for x in compmails[i]]